In [1]:
!pip install feedparser # xml 태그별 정보 추출
!pip install newspaper3k  # 인터넷 신문 기사 분석
!pip install konlpy # 한국어형태소분석기 - 명사만 추출
!pip install lxml[html_clean]

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=0f4b0358affabf50e4b85cb21ce3873246cfdd56869cdfbb8acd62eb3fb36be5
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.1 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=d0a1709e7c465aab973baf7feaa96fba674efc0400eb273aaee2ba793e798ebd
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df4

In [2]:
import feedparser
from newspaper import Article
from konlpy.tag import Okt
from collections import Counter

In [3]:
# 검색할 RSS파일(xml) list
urls = ["http://rss.etnews.com/Section901.xml",
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml",
        "http://rss.etnews.com/Section904.xml"]

In [4]:
# list 링크의 title, link 추출
def crawl_rss(urls):
  array_rss = []
  titles = set()
  for index_url in urls:
    print("[Crawl RSS]", index_url)
    parse_rss = feedparser.parse(index_url)
    for p in parse_rss.entries:
      if p.title in titles:
        continue
      titles.add(p.title)
      array_rss.append({'title':p.title, 'link':p.link})
  return array_rss

list_articles = crawl_rss(urls)
print(len(list_articles))

[Crawl RSS] http://rss.etnews.com/Section901.xml
[Crawl RSS] http://rss.etnews.com/Section902.xml
[Crawl RSS] http://rss.etnews.com/Section903.xml
[Crawl RSS] http://rss.etnews.com/Section904.xml
76


In [5]:
# url의 title, text 추출
def crawl_article(url, language='ko'):
  print('[Crawl Article]', url)
  a = Article(url, language=language)
  a.download()
  a.parse()
  return a.title, a.text

#print(crawl_article(list_articles[0]['link'])[0])

for article in list_articles:
  _, text = crawl_article(article['link'])
  article['text'] = text

[Crawl Article] https://www.etnews.com/20241105000329
[Crawl Article] https://www.etnews.com/20241106000001
[Crawl Article] https://www.etnews.com/20241105000378
[Crawl Article] https://www.etnews.com/20241105000263
[Crawl Article] https://www.etnews.com/20241105000379
[Crawl Article] https://www.etnews.com/20241105000371
[Crawl Article] https://www.etnews.com/20241105000253
[Crawl Article] https://www.etnews.com/20241105000404
[Crawl Article] https://www.etnews.com/20241105000115
[Crawl Article] https://www.etnews.com/20241105000374
[Crawl Article] https://www.etnews.com/20241105000310
[Crawl Article] https://www.etnews.com/20241105000240
[Crawl Article] https://www.etnews.com/20241104000451
[Crawl Article] https://www.etnews.com/20241105000350
[Crawl Article] https://www.etnews.com/20241105000334
[Crawl Article] https://www.etnews.com/20241105000333
[Crawl Article] https://www.etnews.com/20241105000255
[Crawl Article] https://www.etnews.com/20241105000225
[Crawl Article] https://www.

In [6]:
# 긁어온 text에서 명사 추출(키워드, 빈도수)
def get_keyword(text, nKeywords = 10):
  list_keywords = []
  spliter = Okt()
  nouns = spliter.nouns(text) # 명사 추출
  count = Counter(nouns)  # 명사들의 빈도 추출
  for n,c in count.most_common(nKeywords):  # 빈도수 높은 순으로 저장
    list_keywords.append({'keyword':n, 'count':c})
  return list_keywords

for article in list_articles:
  article['keywords'] = get_keyword(article['text'])
print(list_articles[0])

{'title': "위고페어, '원클릭 위조상품 신고서비스' 출시", 'link': 'https://www.etnews.com/20241105000329', 'text': "상품 URL만 올리면 전 세계 온라인마켓 위조품 신고 끝\n\n인공지능(AI) 기반 위조상품 모니터링 및 차단 플랫폼을 운영하는 위고페어가 URL만 입력하면 간편하게 위조상품 신고할 수 있는 '원클릭 위조상품 신고 서비스'를 출시했다고 6일 밝혔다.\n\n\n\n'원클릭 위조상품 신고 서비스'는 위조품 판매 페이지 URL 입력만으로 위조상품 신고가 완료되어 전문지식이 없어도 누구나 쉽게 이용할 수 있다. 일명 '가품' 또는 '짝퉁'이라고 불리는 위조상품이 판매되는 것을 알고 있어도 플랫폼별로 상이한 신고 절차, 언어 장벽, 복잡한 신고 프로세스 등으로 대응을 못해 어려움을 겪던 기업들이 보다 쉽게 브랜드를 보호할 수 있도록 설계됐다. 특히 건별 신고 및 결제가 가능해 기업의 비용 부담을 대폭 줄인 것이 특징이다.\n\n\n\n위고페어 '원클릭 위조상품 신고'서비스는 국내뿐 아니라 다양한 글로벌 쇼핑몰에서 우리 기업의 브랜드 보호를 지원한다. 네이버, 쿠팡 등을 비롯해 알리바바, 테무, 쉬인 등 중국의 주요 이커머스 플랫폼은 물론, 쇼피, 라자다와 같은 동남아 플랫폼, 아마존, 이베이 등 글로벌 플랫폼의 위조상품까지 모두 신고가 가능하다.\n\n\n\n변리사와 지식재산보호원 위조상품 단속팀 출신 전문가들이 직접 신고 프로세스를 관리해 99%의 위조상품 차단 실적을 자랑한다. 기업별 전담 매니저를 배정해 맞춤형 서비스를 신속하게 제공한다.\n\n\n\n김종면 위고페어 CEO는 “위조상품 유통은 기업의 브랜드 이미지 훼손뿐만 아니라 매출 손실, 일자리 감소, 세수 감소 등 국가 경제에도 심각한 피해를 주고 있다”라며 “위고페어는 최첨단 AI 기술과 글로벌 플랫폼 위조품 차단 실무 경험을 보유한 전문 인력들을 바탕으로 우리 기업들의 브랜드를 효과적으로 보호하겠다”고 강조했다.\n\n\n\n위고페어

In [10]:
# 입력한 검색어를 갖고 있는 문서 출력
query = input()

def search_articles(query, list_keywords):
  nWords = 0
  for kw in list_keywords:
    if query == kw['keyword']:
      nWords = kw['count']
  return nWords

articles_with_query = []
for article in list_articles:
  nQuery = search_articles(query, article['keywords'])
  if nQuery > 0:
    articles_with_query.append([article['title'], article['link'], nQuery])

articles_with_query.sort(key=lambda x:x[2], reverse=True)

for article in articles_with_query:
  print('[TF]', article[2], '[Title]', article[0])
  print('[URL]', article[1])

기업
[TF] 20 [Title] 비즈니스 혁신하는 통합 솔루션 네이버웍스
[URL] https://www.etnews.com/20241106000017
[TF] 10 [Title] 전남정보문화산업진흥원 입주기업 '남도소반', 말레이시아 기업과 수출 MOU 체결
[URL] https://www.etnews.com/20241106000025
[TF] 6 [Title] 위고페어, '원클릭 위조상품 신고서비스' 출시
[URL] https://www.etnews.com/20241105000329
[TF] 4 [Title] 사람인, 개인정보보호 수준 향상…강화된 안전조치 도입
[URL] https://www.etnews.com/20241106000018
[TF] 3 [Title] 인디제이, '눈치 AI' 기반 차량용 인포테인먼트 비서 서비스 출시
[URL] https://www.etnews.com/20241106000011
[TF] 2 [Title] 6.7대1 경쟁률 뚫고 경남 제조기업 R&D 역량 과시
[URL] https://www.etnews.com/20241106000016
